# Sarcasm-aware sentiment playground

This notebook demonstrates how the hybrid sentiment pipeline reacts to sarcasm-heavy Reddit comments.
It also summarizes lightweight research takeaways to guide future improvements.

## Research snapshot
- **Specialized irony classifiers** (e.g., `cardiffnlp/twitter-roberta-base-irony`) outperform generic sentiment models on Reddit-like data because they are trained on short, informal posts.
- **Context cues** that boost sarcasm recall include emoji (🙃, 😂), exaggerated punctuation (`???!!!`), hyperbole words ("totally", "best ever"), and contradiction between sentiment words and subject ("Great, another crash").
- **Confidence-aware blending** is safer than hard flipping: when sarcasm probability is high, dampen sentiment magnitude instead of inverting it to avoid overcorrection.
- **Human-in-the-loop evaluation** with real Reddit snippets remains essential; many sarcastic comments are community- or thread-specific.


In [ ]:
import sys
from pprint import pprint

# Ensure the repository code is on the path for notebook execution
if ".." not in sys.path:
    sys.path.append("..")

from app.services.hybrid_sentiment import HybridSentimentService

# Configure the hybrid service with sarcasm dampening enabled
service = HybridSentimentService(
    use_llm=True,
    dual_model_strategy=True,
    enable_sarcasm_detection=True,
    sarcasm_threshold=0.6,
    sarcasm_dampening_factor=0.6,
    strong_llm_threshold=0.25,
)

print("Service configuration:")
pprint(service.get_service_info())


In [ ]:
sample_comments = [
    "Sure, because this stock only goes up... 🤡",
    "Absolutely love losing money on every dip, best feeling ever!",
    "Solid earnings and a clean balance sheet. I'm bullish here.",
    "Yeah right, management will totally deliver this time 🙃",
    "This is fine.",
]

def run_examples(comments: list[str]):
    rows = []
    for text in comments:
        details = service.analyze_with_details(text)
        sarcasm = details["sarcasm"]
        rows.append(
            {
                "text": text,
                "raw": round(details["raw_score"], 3),
                "adjusted": round(details["adjusted_score"], 3),
                "label": details["label"],
                "sarcasm_prob": None if not sarcasm else round(sarcasm.probability, 3),
                "sarcastic": None if not sarcasm else sarcasm.is_sarcastic,
            }
        )

    for row in rows:
        print("-" * 80)
        print(row["text"])
        print(
            f"raw={row['raw']} adjusted={row['adjusted']} label={row['label']} sarcasm_prob={row['sarcasm_prob']} sarcasm?={row['sarcastic']}"
        )

run_examples(sample_comments)


## Try your own comments

Edit `sample_comments` above or create a new list below to quickly compare how sarcasm
changes the adjusted sentiment score.


In [ ]:
custom_comments = [
    "Oh great, another earnings miss. Totally thrilled.",
    "Huge beat and strong guidance, love to see it.",
]

run_examples(custom_comments)
